In [35]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")

import nltk
from thefuzz import fuzz
from thefuzz import process

In [2]:
%%time
df=pd.read_csv('wiki_movie_plots_deduped.csv')
df['Genre'].nunique()

Wall time: 1.36 s


2265

In [3]:

df['Genre']= df['Genre'].str.replace("unknown", "")


In [4]:
dax=df['Genre'].value_counts().rename_axis('Genres').reset_index(name='counts')
dax[:30]

,Genres,counts
0,,6083
1,drama,5964
2,comedy,4379
3,horror,1167
4,action,1098
5,thriller,966
6,romance,923
7,western,865
8,crime,568
9,adventure,526


In [5]:
[i for i in dax['Genres'].to_list() if len(i.split())<2][1:18]

['drama',
 'comedy',
 'horror',
 'action',
 'thriller',
 'romance',
 'western',
 'crime',
 'adventure',
 'musical',
 'mystery',
 'war',
 'animation',
 'sci-fi',
 'family',
 'fantasy',
 'animated']

In [8]:
choices = [i for i in dax['Genres'].to_list() if len(i.split())<2][1:18]


In [9]:
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])
        

In [10]:
%%time
for i in range(10,len(dax['Genres'].to_list()),10):
    inputt=dax['Genres'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)


Wall time: 3min 26s


In [19]:
df['Genre']=df['Genre'].apply(lambda x: x.strip())

In [20]:
[[row['Genres'],row['counts']] for i,row in dax.iterrows() if row['Genres'] in choices ]

[['', 6083],
 ['drama', 5964],
 ['comedy', 4379],
 ['horror', 1167],
 ['action', 1098],
 ['thriller', 966],
 ['romance', 923],
 ['western', 865],
 ['crime', 568],
 ['adventure', 526],
 ['musical', 467],
 ['science fiction', 418],
 ['film noir', 345],
 ['mystery', 310],
 ['war', 273],
 ['animation', 264],
 ['sci-fi', 221],
 ['family', 217],
 ['fantasy', 204],
 ['animated', 195],
 ['biography', 136],
 ['anime', 112],
 ['suspense', 104],
 ['social', 82],
 ['historical', 77],
 ['documentary', 73],
 ['serial', 71],
 ['biopic', 57],
 ['superhero', 48],
 ['martial arts', 46],
 ['short', 30],
 [' ', 28],
 ['tokusatsu', 28],
 ['spy', 27],
 ['dramedy', 24],
 ['sports', 23],
 ['mythology', 22],
 ['biographical', 20],
 ['folklore', 17],
 ['masala', 16],
 ['rom com', 14],
 ['propaganda', 13],
 ['kaiju', 13],
 ['devotional', 12],
 ['slapstick', 12],
 ['slasher', 12],
 ['disaster film', 11],
 ['swashbuckler', 10],
 ['epic', 10],
 ['wuxia', 9],
 ['satire', 8],
 ['parody', 8],
 ['children', 7],
 ['dete

In [21]:
replace_dict={
   "science_friction":"sci-fi",
    "sci fi":"sci-fi",
    "science friction":"sci-fi",
    "amine":"animated",
    "animation":"animated",
    "cartoon":"animated",
    "tokusatsu": "animated",
    "superhero": "animated",
    "suspense": "thriller",
    "dramedy": "drama",
    "blaxploitation": "exploitation",
    "biopic": "biography",
    "bio-pic": "biography",
    "biographical": "biography",
    "kung fu": "martial_arts",
    "martial arts": "martial_arts",
    "kung fu": "martial_arts",
    "film": "",
    "neo-noir": "noir",
    "devotional": "religious",
    "children": "teen",
    "porno": "adult",
    "history": "historical",
    "rom com": "romance-comedy",
    "rom-com": "romance-comedy",
    "love": "romance",
}

In [22]:
for key,items in replace_dict.items():
    df['Genre']= df['Genre'].str.replace(key,items)

In [23]:
dax=df['Genre'].value_counts().rename_axis('Genres').reset_index(name='counts')
choices = [i for i in dax['Genres'].to_list() if len(i.split())<2][0:18]
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])
        


In [24]:
%%time
for i in range(10,len(dax['Genres'].to_list()),10):
    inputt=dax['Genres'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)

Wall time: 2min 46s


In [25]:
choices_final=[row['Genres'] for i,row in dax.iterrows() if row['Genres'] in choices if row['counts']>50]

In [26]:
choices_final

['',
 'drama',
 'comedy',
 'horror',
 'action',
 'thriller',
 'romance',
 'western',
 'crime',
 'animated',
 'adventure',
 'musical',
 'science fiction',
 'noir',
 'mystery',
 'war',
 'sci-fi',
 'biography',
 'family',
 'fantasy',
 'anime',
 'social',
 'historical',
 'documentary',
 'serial',
 'martial_arts']

In [27]:

def genre_list(x):
    
    if x in choices_final:
        return x
    elif process.extractBests(x, choices_final,score_cutoff=80,limit=20) !=[]:
        a=process.extractBests(x, choices_final,score_cutoff=80,limit=20)
        return ','.join([a[i][0] for i in range(len(a))])
    else:
        return ''

In [28]:
genre_list('drama romance')

'drama,romance'

In [29]:
df['Genre_1']=df['Genre']

In [30]:
df['Genre_1']=df['Genre_1'].apply(lambda x: genre_list(x))

In [31]:
def genre(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page', 'Plot', 'Genre_1','Genre_updated'])
    for i,row in data.iterrows():
        if len(row['Genre_1'].split(','))>1:
            for i in row['Genre_1'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Genre_updated']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Genre_updated']=df['Genre_1']
            df1=pd.concat([df1,df])
            
    return df1.reset_index(drop=True)

In [32]:
%%time
new_df=genre(df)

Wall time: 3min 55s


In [33]:
new_df['Genre_updated'].value_counts()

drama              9432
comedy             7266
                   6777
action             2446
romance            1973
thriller           1916
crime              1597
horror             1576
western             974
musical             950
animated            894
adventure           868
family              634
war                 600
science fiction     582
fantasy             528
mystery             481
biography           457
sci-fi              389
noir                356
historical          257
social              223
anime               143
documentary         130
martial_arts        129
serial               83
Name: Genre_updated, dtype: int64

In [34]:
new_df.drop(['Genre_1'],axis=1).to_csv('wiki_movie_plots_deduped_cleaned_genre.csv',index=False)

In [37]:
new_df[new_df['Genre']!=new_df['Genre_updated']]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_1,Genre_updated
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...,,
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,"action,western,crime",action
12,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,"action,western,crime",western
13,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,"action,western,crime",crime
14,1906,Kathleen Mavourneen,American,Edwin S. Porter,NaN,short,https://en.wikipedia.org/wiki/Kathleen_Mavourn...,Irish villager Kathleen is a tenant of Captain...,,
...,...,...,...,...,...,...,...,...,...,...
41646,2011,Zephyr,Turkish,Belma Baş,"Şeyma Uzunlar, Vahide Gördüm & Sevinç Baş",comedy-drama,https://en.wikipedia.org/wiki/Zephyr_(film),"Zephyr is a strong-willed little girl, spendin...","drama,comedy",drama
41647,2011,Zephyr,Turkish,Belma Baş,"Şeyma Uzunlar, Vahide Gördüm & Sevinç Baş",comedy-drama,https://en.wikipedia.org/wiki/Zephyr_(film),"Zephyr is a strong-willed little girl, spendin...","drama,comedy",comedy
41652,2013,Particle (film),Turkish,Erdem Tepegöz,"Jale Arıkan, Rüçhan Caliskur, Özay Fecht, Remz...",drama,https://en.wikipedia.org/wiki/Particle_(film),"Zeynep lost her job at weaving factory, and he...",drama,drama
41659,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,comedy,comedy


In [55]:
new_df[(new_df['Genre_updated']=='') & (new_df['Genre']!=new_df['Genre_updated'])]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_1,Genre_updated
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...,,
14,1906,Kathleen Mavourneen,American,Edwin S. Porter,NaN,short,https://en.wikipedia.org/wiki/Kathleen_Mavourn...,Irish villager Kathleen is a tenant of Captain...,,
38,1911,Baseball and Bloomers,American,Unknown,"William Garwood, Marguerite Snow",silent sports,https://en.wikipedia.org/wiki/Baseball_and_Blo...,"Though the film is presumed lost, a synopsis s...",,
117,1915,Birth of a Nation,American,D. W. Griffith,"Lillian Gish, Mae Marsh",epic,https://en.wikipedia.org/wiki/Birth_of_a_Nation,The film follows two juxtaposed families. One ...,,
180,1917,Enlighten Thy Daughter,American,Ivan Abramson,NaN,sexual hygiene/exploitation,https://en.wikipedia.org/wiki/Enlighten_Thy_Da...,The exploitation/sexual hygiene film warns aga...,,
...,...,...,...,...,...,...,...,...,...,...
41144,1993,Madame Aema 8,South_Korean,Suk Do-won,Ru Mina\r\nKang Eun-ah,ero,https://en.wikipedia.org/wiki/Madame_Aema_8,Following the multiple-Aema theme started in M...,,
41145,1993,Madame Aema 9,South_Korean,Kim Sung-su,Jin Ju-hui,ero,https://en.wikipedia.org/wiki/Madame_Aema_9,Aema is a bored housewife married to a success...,,
41150,1994,Madame Aema 10,South_Korean,Suk Do-won,O No-a,ero,https://en.wikipedia.org/wiki/Madame_Aema_10,"This time in the Madame Aema series, Aema leav...",,
41153,1995,Madame Aema 11,South_Korean,Joe Moung-hwa,Lee Da-yeon,ero,https://en.wikipedia.org/wiki/Madame_Aema_11,"In this episode in the Madame Aema series, Aem...",,
